# Subduction teeth overview

This Notebook creates subduction teeth on a pyGPlates reconstructed timeseries. Before running this Notebook, run the [`resolve_topologies.py`](https://github.com/EarthByte/PlateTectonicTools/blob/master/ptt/resolve_topologies.py) script in [PlateTectonicTools](https://github.com/EarthByte/PlateTectonicTools) with command line options to output the required shapefiles, or run the Python code below if you have PTT installed.

```python
# resolve topologies
for reconstruction_time in reconstruction_times:
    ptt.resolve_topologies.resolve_topologies(rotation_model,
                                              topology_filenames,
                                              reconstruction_time,
                                              'reconstructed_plate_geometries/resolve_topologies/',
                                              output_filename_extension='shp',
                                              anchor_plate_id=0)
```
keep note of the output shapefile directory.

#### Output -- ensure `snapshots` folder exists!

All your timesteps are output to this folder.

- `my_reconstruction_0001.png`
- `my_reconstruction_0002.png` etc.

Now you can turn all of the snapshots into an animation with FFMPEG:

```sh
cat $(ls -r my_reconstruction_*) | ffmpeg -f image2pipe -r 8 -i - -vcodec libx264 timeseries.mp4
```

This command reverses the order of the images and feeds it to FFMPEG where a mp4 video is encoded at 8 fps using the x264 codec.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

In [2]:
shapefile_dir = "../reconstructed_plate_geometries/"

# Define the time range.
min_time = 0
max_time = 200
averaging_time_window = 1
reconstruction_times = np.arange(max_time, min_time, -averaging_time_window) # step forward in time

extent_globe = [-180,180,-90,90]


In [3]:
def get_valid_geometries(shape_filename):
    """
    Return valid geometries within a shapefile
    
    Shapefiles are incredibly annoying. In a lot of cases the boundary is corrupted which means
    filled shapes leak out into the rest of the domain. Sometimes this can be mitigated by setting
    a buffer, sometimes not. Using the boundaries with no fill works most of the time.
    
    Caveat Emptor!
    """
    shp_geom = cfeature.shapereader.Reader(shape_filename).geometries()
    geometries = []
    for record in shp_geom:
        if record.is_valid:
            geometries.append(record.boundary)
#             geometries.append(record.buffer(0.))
    return geometries

In [4]:
# for interpolating from icosohedral mesh onto a grid

nlon = 1441
nlat = 721

proj = ccrs.Mollweide(142)


# global parameters
tol = 0.01
averaging_time_window = 1

## Subduction teeth generator functions

In [5]:
# shp_subdL.geometries()

def tesselate_triangles(shapefilename, tesselation_radians, triangle_base_length, triangle_aspect=1.0):
    """
    Place subduction teeth along line segments within a MultiLineString shapefile
    
    Parameters
    ----------
        shapefilename  : str  path to shapefile
        tesselation_radians : float
        triangle_base_length : float  length of base
        triangle_aspect : float  aspect ratio
        
    Returns
    -------
        X_points : (n,3) array of triangle x points
        Y_points : (n,3) array of triangle y points
    """

    import shapefile
    shp = shapefile.Reader(shapefilename)

    tesselation_degrees = np.degrees(tesselation_radians)
    triangle_pointsX = []
    triangle_pointsY = []

    for i in range(len(shp)):
        pts = np.array(shp.shape(i).points)

        cum_distance = 0.0
        for p in range(len(pts) - 1):

            A = pts[p]
            B = pts[p+1]

            AB_dist = B - A
            AB_norm = AB_dist / np.hypot(*AB_dist)
            cum_distance += np.hypot(*AB_dist)

            # create a new triangle if cumulative distance is exceeded.
            if cum_distance >= tesselation_degrees:

                C = A + triangle_base_length*AB_norm

                # find normal vector
                AD_dist = np.array([AB_norm[1], -AB_norm[0]])
                AD_norm = AD_dist / np.linalg.norm(AD_dist)

                C0 = A + 0.5*triangle_base_length*AB_norm

                # project point along normal vector
                D = C0 + triangle_base_length*triangle_aspect*AD_norm

                triangle_pointsX.append( [A[0], C[0], D[0]] )
                triangle_pointsY.append( [A[1], C[1], D[1]] )

                cum_distance = 0.0

    shp.close()
    return np.array(triangle_pointsX), np.array(triangle_pointsY)

def tesselate_arrows(shapefilename, tesselation_radians, arrow_length, buffer=0.0, near_points=None, **kwargs):
    
    import shapefile
    shp = shapefile.Reader(shapefilename)
    
    if near_points is None:
        d = np.array(0.0)
    else:
        from scipy.spatial import cKDTree
        tree = cKDTree(near_points)
        
    d_tol = 0.5

    tesselation_degrees = np.degrees(tesselation_radians)
    arrow_pointsX = []
    arrow_pointsY = []
    
    for i in range(len(shp)):
        pts = np.array(shp.shape(i).points)
        
        cum_distance = 0.0
        for p in range(len(pts) - 1):
            
            A = pts[p]
            B = pts[p+1]
            
            AB_dist = B - A
            AB_norm = AB_dist / np.linalg.norm(AB_dist)
            cum_distance += np.linalg.norm(AB_dist)
            
            if near_points is not None:
                d, idx = tree.query(A)
            
            if cum_distance >= tesselation_degrees and d < d_tol:
                
                AD_dist = np.array([AB_norm[1], -AB_norm[0]])
                AD_norm = AD_dist / np.linalg.norm(AD_dist)
                
                dC = arrow_length*AD_norm
                
                arrow_pointsX.append( [A[0], dC[0]] )
                arrow_pointsY.append( [A[1], dC[1]] )
                
    return np.array(arrow_pointsX), np.array(arrow_pointsY)

## Reconstruct CO2 in plate

Import `CO2_conveyor_belt.nc` and save a figure with CO2 fields overlain with reconstructed coastline polygons to file.

In [6]:
xticks = list(np.arange(-180,180+20,20))
yticks = list(np.arange(-90, 90+10, 10))

In [18]:

for i, reconstruction_time in enumerate(reconstruction_times[:1]):
    
    ## Shapefiles and feature geometries
    # read ridges
    shp_ridges = shpreader.Reader(shapefile_dir+"resolve_topologies/ridge_transform_boundaries_{:.2f}Ma.shp".format(reconstruction_time)).geometries()
    ft_ridges  = cfeature.ShapelyFeature(shp_ridges, ccrs.PlateCarree())

    # read subduction boundaries
    shp_subd = shpreader.Reader(shapefile_dir+"resolve_topologies/subduction_boundaries_{:.2f}Ma.shp".format(reconstruction_time)).geometries()
    ft_subd  = cfeature.ShapelyFeature(shp_subd, ccrs.PlateCarree())

    # read coastlines
    shp_coastlines = get_valid_geometries(shapefile_dir+"coastline/coastlines_{:04.0f}.shp".format(reconstruction_time))
    ft_coastlines  = cfeature.ShapelyFeature(shp_coastlines, ccrs.PlateCarree())
    
    # read subduction boundaries
    shp_subdL = shpreader.Reader(shapefile_dir+"resolve_topologies/subduction_boundaries_sL_{:.2f}Ma.shp".format(reconstruction_time)).geometries()
    ft_subdL  = cfeature.ShapelyFeature(shp_subdL, ccrs.PlateCarree())

    shp_subdR = shpreader.Reader(shapefile_dir+"resolve_topologies/subduction_boundaries_sR_{:.2f}Ma.shp".format(reconstruction_time)).geometries()
    ft_subdR  = cfeature.ShapelyFeature(shp_subdR, ccrs.PlateCarree())

    


    # create figure
    fig = plt.figure(figsize=(8,8), dpi=300)
    ax1 = plt.axes(projection=proj)
    
    # replace this dummy array with something else
    ax1.imshow(np.array([[0,0],[0,0]]), extent=[-180,180,-180,180],
               vmin=-1, vmax=5, cmap=plt.cm.Greys, transform=ccrs.PlateCarree())
    
    # add coastline + continents
    ax1.add_feature(ft_ridges, facecolor='none', edgecolor='k', linewidth=0.5, zorder=4)
    ax1.add_feature(ft_coastlines, facecolor='none', edgecolor='k', linewidth=0.25, zorder=5)
    ax1.add_feature(ft_subd, facecolor='none', edgecolor='k', linewidth=1, zorder=4)
    
    # add Subduction Teeth
    subd_xL, subd_yL = tesselate_triangles(
        shapefile_dir+"resolve_topologies/subduction_boundaries_sL_{:.2f}Ma.shp".format(reconstruction_time),0.1, 1.,-1.0)
    subd_xR, subd_yR = tesselate_triangles(
        shapefile_dir+"resolve_topologies/subduction_boundaries_sR_{:.2f}Ma.shp".format(reconstruction_time),0.1, 1., 1.0)

    for tX, tY in zip(subd_xL, subd_yL):
        triangle_xy_points = np.c_[tX, tY]
        patch = plt.Polygon(triangle_xy_points, color='k', transform=ccrs.PlateCarree(), zorder=6)
        ax1.add_patch(patch)
    for tX, tY in zip(subd_xR, subd_yR):
        triangle_xy_points = np.c_[tX, tY]
        patch = plt.Polygon(triangle_xy_points, color='k', transform=ccrs.PlateCarree(), zorder=6)
        ax1.add_patch(patch)


    ax1.gridlines(xlocs=xticks, ylocs=yticks, alpha=0.5, linestyle=':', linewidth=0.5)
    fig.savefig('./snapshots/my_reconstruction_{:04d}.png'.format(int(reconstruction_time)),
                dpi=300, bbox_inches='tight')

    plt.close(fig)
    
    print("reconstruction time = {} Ma".format(reconstruction_time))



reconstruction time = 200 Ma


Now you can turn all of the snapshots into an animation with FFMPEG:

```sh
cat $(ls -r my_reconstruction_*) | ffmpeg -f image2pipe -r 8 -i - -vcodec libx264 timeseries.mp4
```

This command reverses the order of the images and feeds it to FFMPEG where a mp4 video is encoded at 8 fps using the x264 codec.